# RNN con el modelo Bidireccional

In [1]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import GlobalMaxPooling1D
import numpy as np
from keras.layers import Embedding, SimpleRNN, Dense, Bidirectional

print("listo")

listo


## definimos el numero de secuencias, para un entrenamiento de secuencias largas y cortas y los dividimos en valores de entrenamiento y testeo

In [2]:
max_features = 5000  # Number of words to consider as features
max_len_short = 100  # Maximum sequence length for short sequences
max_len_long = 500   # Maximum sequence length for long sequences

##y_train = np.expand_dims(y_train, axis=1)
##y_test = np.expand_dims(y_test, axis=1)

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

print("listo")

listo


## dividimos en Padings los datos de entrenamiento y testeo

In [3]:
x_train_short = sequence.pad_sequences(x_train, maxlen=max_len_short)
x_test_short = sequence.pad_sequences(x_test, maxlen=max_len_short)

x_train_long = sequence.pad_sequences(x_train, maxlen=max_len_long)
x_test_long = sequence.pad_sequences(x_test, maxlen=max_len_long)
print("listo")

listo


## definimos los modelos que vamos a utilizar mediante estas funciones ya sea modelo Elman Jordan o bidireccional

In [4]:
def build_elman_rnn_model():
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(SimpleRNN(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

def build_jordan_rnn_model():
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(SimpleRNN(32, activation='relu', return_sequences=True))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(1, activation='sigmoid'))
    return model

def build_bidirectional_rnn_model():
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(Bidirectional(SimpleRNN(32, activation='relu')))
    model.add(Dense(1, activation='sigmoid'))
    return model
print("listo")

listo


## definimos una función de entrenamiento con su optimizador y función de perdida que evalua al modelo mediante los valores de perdida y el accuracy, podemos tambien configurar el numero de Epocas y el tamaño del batch

In [5]:
def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.2)
    loss, accuracy = model.evaluate(x_test, y_test)
    return loss, accuracy, history
print("listo")

listo


## Procedemos a seleccionar el modelo y entrenarlo 

In [6]:
print("\nTraining RNN model on short sequences:")
rnn_model_short = build_bidirectional_rnn_model()
loss_short, accuracy_short, history_short = train_and_evaluate_model(
    rnn_model_short, x_train_short, y_train, x_test_short, y_test
)
print("listo")


Training RNN model on short sequences:
Epoch 1/5
157/157 [==============================] - 14s 71ms/step - loss: 0.6233 - accuracy: 0.6334 - val_loss: 0.4612 - val_accuracy: 0.7774
Epoch 2/5
157/157 [==============================] - 10s 65ms/step - loss: 0.3510 - accuracy: 0.8468 - val_loss: 0.3586 - val_accuracy: 0.8422
Epoch 3/5
157/157 [==============================] - 11s 68ms/step - loss: 0.2725 - accuracy: 0.8905 - val_loss: 0.3925 - val_accuracy: 0.8256
Epoch 4/5
157/157 [==============================] - 11s 71ms/step - loss: 0.2334 - accuracy: 0.9075 - val_loss: 0.3621 - val_accuracy: 0.8466
Epoch 5/5
782/782 [==============================] - 11s 14ms/step - loss: 0.3914 - accuracy: 0.8441
listo


In [7]:
print("\nTraining Elman RNN model on long sequences:")
rnn_model_long = build_bidirectional_rnn_model()
loss_long, accuracy_long, history_long = train_and_evaluate_model(
    rnn_model_long, x_train_long, y_train, x_test_long, y_test
)
print("listo")


Training Elman RNN model on long sequences:
Epoch 1/5
157/157 [==============================] - 51s 304ms/step - loss: 0.6336 - accuracy: 0.6330 - val_loss: 0.5062 - val_accuracy: 0.7712
Epoch 2/5
157/157 [==============================] - 48s 304ms/step - loss: 0.3696 - accuracy: 0.8461 - val_loss: 0.3577 - val_accuracy: 0.8486
Epoch 3/5
157/157 [==============================] - 49s 310ms/step - loss: 0.2867 - accuracy: 0.8835 - val_loss: 0.3205 - val_accuracy: 0.8700
Epoch 4/5
157/157 [==============================] - 49s 310ms/step - loss: 0.2436 - accuracy: 0.9060 - val_loss: 0.3214 - val_accuracy: 0.8660
Epoch 5/5
782/782 [==============================] - 36s 46ms/step - loss: 0.3239 - accuracy: 0.8646
listo


In [8]:
print("\nResults on Short Sequences:")
print(f"Loss: {loss_short:.4f}, Accuracy: {accuracy_short:.4f}")

print("\nResults on Long Sequences:")
print(f"Loss: {loss_long:.4f}, Accuracy: {accuracy_long:.4f}")


Results on Short Sequences:
Loss: 0.3914, Accuracy: 0.8441

Results on Long Sequences:
Loss: 0.3239, Accuracy: 0.8646


# Conclusión

## el acuracy en los tres casos es virtualmente igual

## Elman 

Results on Short Sequences:
Loss: 0.3871, Accuracy: 0.8386

Results on Long Sequences:
Loss: 0.3290, Accuracy: 0.8616

## Jordan 

Results on Short Sequences:
Loss: 0.4302, Accuracy: 0.8244

Results on Long Sequences:
Loss: 0.3431, Accuracy: 0.8632

## Bidireccional

Results on Short Sequences:
Loss: 0.3914, Accuracy: 0.8441

Results on Long Sequences:
Loss: 0.3239, Accuracy: 0.8646


### La unica diferencia es que en el modelo Jordan había que retornar la secuencia y generaba un error si no se le colocaba false, para solucionar este error se decidió reducir la secuencia de las palabras  a una sola presentación antes de pasarlas a la capa de clasificación final, para esto se importó GlobalMaxPooling1D.


